# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [62]:
problem = '''You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened. 
Please formulate this as a mathematical optimization model.'''

## 2. Generate the mathematical model

In [63]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [64]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Facility Location

**Sets and Indices:**

*  *I*: Set of customers
* *J*: Set of potential facility locations

**Parameters:**

* *f<sub>j</sub>*: Fixed annual activation cost of facility *j* ∈ *J*
* *cap<sub>j</sub>*: Maximum annual service volume of facility *j* ∈ *J*
* *d<sub>i</sub>*: Annual demand of customer *i* ∈ *I*
* *c<sub>ij</sub>*: Transportation cost per unit to serve customer *i* ∈ *I* from facility *j* ∈ *J*

**Decision Variables:**

* *x<sub>ij</sub>*: Amount of demand of customer *i* ∈ *I* served by facility *j* ∈ *J*
* *y<sub>j</sub>*: Binary variable indicating whether facility *j* ∈ *J* is opened (1) or not (0)

**Objective Function:**

Minimize the total cost, which includes fixed activation costs and transportation costs:

```
Minimize Σ<sub>j∈J</sub> f<sub>j</sub>y<sub>j</sub> + Σ<sub>i∈I</sub> Σ<sub>j∈J</sub> c<sub>ij</sub>x<sub>ij</sub> 
```

**Constraints:**

1. **Demand satisfaction:** Each customer's demand must be met:
   ```
   Σ<sub>j∈J</sub> x<sub>ij</sub> = d<sub>i</sub>, ∀ i ∈ I
   ```

2. **Capacity constraint:** Each facility cannot exceed its maximum service volume:
   ```
   Σ<sub>i∈I</sub> x<sub>ij</sub> ≤ cap<sub>j</sub>y<sub>j</sub>, ∀ j ∈ J
   ```

3. **Service from open facilities:** Customers can only be serviced by open facilities:
   ```
   x<sub>ij</sub> ≤ d<sub>i</sub>y<sub>j</sub>, ∀ i ∈ I, ∀ j ∈ J
   ```

4. **Binary constraint:**
   ```
   y<sub>j</sub> ∈ {0, 1}, ∀ j ∈ J 
   ```

This model represents a mixed-integer linear program, which can be solved using optimization software to determine the optimal locations for facilities and the allocation of customers to facilities to minimize overall costs. 


## 3. Generate the pyomo code

In [65]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [66]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with your actual data)
customers = ['A', 'B', 'C']
facilities = ['1', '2', '3']
fixed_cost = {'1': 100, '2': 150, '3': 120}  
capacity = {'1': 500, '2': 600, '3': 550} 
demand = {'A': 200, 'B': 350, 'C': 250}
transportation_cost = {
    ('A', '1'): 2, ('A', '2'): 3, ('A', '3'): 2.5,
    ('B', '1'): 2.8, ('B', '2'): 1.5, ('B', '3'): 2,
    ('C', '1'): 1.8, ('C', '2'): 2.2, ('C', '3'): 1.7
}

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=customers) # set of customers
model.J = pyo.Set(initialize=facilities) # set of potential facilities

# Parameters
model.f = pyo.Param(model.J, initialize=fixed_cost) # fixed cost of opening a facility
model.cap = pyo.Param(model.J, initialize=capacity) # capacity of a facility
model.d = pyo.Param(model.I, initialize=demand) # demand of each customer
model.c = pyo.Param(model.I, model.J, initialize=transportation_cost) # cost of shipping from i to j

# Variables
model.x = pyo.Var(model.I, model.J, bounds=(0, None)) # amount of demand served by facility j for customer i
model.y = pyo.Var(model.J, within=pyo.Binary) # indicator variable: 1 if facility j is opened, 0 otherwise

# Objective Function
model.obj = pyo.Objective(expr=sum(model.f[j]*model.y[j] for j in model.J) + 
                           sum(model.c[i,j]*model.x[i,j] for i in model.I for j in model.J), sense=pyo.minimize)

# Constraints
model.demand_constraint = pyo.Constraint(model.I, rule=lambda model, i: sum(model.x[i,j] for j in model.J) == model.d[i]) # demand satisfaction
model.capacity_constraint = pyo.Constraint(model.J, rule=lambda model, j: sum(model.x[i,j] for i in model.I) <= model.cap[j]*model.y[j]) # capacity constraint
model.service_constraint = pyo.Constraint(model.I, model.J, rule=lambda model, i, j: model.x[i,j] <= model.d[i]*model.y[j]) # service from open facilities only

# Solve
solver = pyo.SolverFactory('glpk') # You can choose other solvers like 'gurobi', 'cplex', etc.
results = solver.solve(model)

# Print Results
print(results) 
print("Optimal Solution:")
for j in model.J:
    if pyo.value(model.y[j]) == 1:
        print(f"Facility {j} is open.")
        for i in model.I:
            if pyo.value(model.x[i,j]) > 0:
                print(f"  Serving {pyo.value(model.x[i,j])} units of demand for customer {i}.") 
```

In this code, I've used sample data for the sets, parameters, and transportation costs. You should replace these with your own data. The code includes comments to indicate where the sample data is used. 


## 4. Input problem data and try running the generated code

In [69]:
import pyomo.environ as pyo

# Sample Data (Replace with your actual data)
customers = [1,2,3,4,5]
facilities = [1, 2, 3]
fixed_cost = {1: 1000, 2: 1000, 3: 1000}  
capacity = {1: 500, 2: 500, 3: 500} 
demand = {1: 80, 2: 270, 3: 250,4:160,5:180}
transportation_cost = {
    (1, 1): 4,  (1, 2): 6,  (1, 3): 9,
    (2, 1): 5,  (2, 2): 4,  (2, 3): 7,
    (3, 1): 6,  (3, 2): 3,  (3, 3): 4,
    (4, 1): 8,  (4, 2): 5,  (4, 3): 3,
    (5, 1): 10, (5, 2): 8,  (5, 3): 4,
}
     

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=customers) # set of customers
model.J = pyo.Set(initialize=facilities) # set of potential facilities

# Parameters
model.f = pyo.Param(model.J, initialize=fixed_cost) # fixed cost of opening a facility
model.cap = pyo.Param(model.J, initialize=capacity) # capacity of a facility
model.d = pyo.Param(model.I, initialize=demand) # demand of each customer
model.c = pyo.Param(model.I, model.J, initialize=transportation_cost) # cost of shipping from i to j

# Variables
model.x = pyo.Var(model.I, model.J, bounds=(0, None)) # amount of demand served by facility j for customer i
model.y = pyo.Var(model.J, within=pyo.Binary) # indicator variable: 1 if facility j is opened, 0 otherwise

# Objective Function
model.obj = pyo.Objective(expr=sum(model.f[j]*model.y[j] for j in model.J) + 
                           sum(model.c[i,j]*model.x[i,j] for i in model.I for j in model.J), sense=pyo.minimize)

# Constraints
model.demand_constraint = pyo.Constraint(model.I, rule=lambda model, i: sum(model.x[i,j] for j in model.J) == model.d[i]) # demand satisfaction
model.capacity_constraint = pyo.Constraint(model.J, rule=lambda model, j: sum(model.x[i,j] for i in model.I) <= model.cap[j]*model.y[j]) # capacity constraint
model.service_constraint = pyo.Constraint(model.I, model.J, rule=lambda model, i, j: model.x[i,j] <= model.d[i]*model.y[j]) # service from open facilities only

# Solve
solver = pyo.SolverFactory('glpk') # You can choose other solvers like 'gurobi', 'cplex', etc.
results = solver.solve(model)

# Print Results
print(results) 
print("Optimal Solution:")
for j in model.J:
    if pyo.value(model.y[j]) == 1:
        print(f"Facility {j} is open.")
        for i in model.I:
            if pyo.value(model.x[i,j]) > 0:
                print(f"  Serving {pyo.value(model.x[i,j])} units of demand for customer {i}.") 


Problem: 
- Name: unknown
  Lower bound: 5610.0
  Upper bound: 5610.0
  Number of objectives: 1
  Number of constraints: 23
  Number of variables: 18
  Number of nonzeros: 63
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.040258169174194336
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal Solution:
Facility 2 is open.
  Serving 80.0 units of demand for customer 1.
  Serving 270.0 units of demand for customer 2.
  Serving 150.0 units of demand for customer 3.
Facility 3 is open.
  Serving 7.78943796698235e-30 units of demand for customer 2.
  Serving 100.0 units of demand for customer 3.
  Serving 160.0 units of demand for customer 4.
  Serving 180.0 units of demand for customer 5.


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.

In [70]:
print(response.text)

## Mathematical Optimization Model for Facility Location

**Sets and Indices:**

*  *I*: Set of customers
* *J*: Set of potential facility locations

**Parameters:**

* *f<sub>j</sub>*: Fixed annual activation cost of facility *j* ∈ *J*
* *cap<sub>j</sub>*: Maximum annual service volume of facility *j* ∈ *J*
* *d<sub>i</sub>*: Annual demand of customer *i* ∈ *I*
* *c<sub>ij</sub>*: Transportation cost per unit to serve customer *i* ∈ *I* from facility *j* ∈ *J*

**Decision Variables:**

* *x<sub>ij</sub>*: Amount of demand of customer *i* ∈ *I* served by facility *j* ∈ *J*
* *y<sub>j</sub>*: Binary variable indicating whether facility *j* ∈ *J* is opened (1) or not (0)

**Objective Function:**

Minimize the total cost, which includes fixed activation costs and transportation costs:

```
Minimize Σ<sub>j∈J</sub> f<sub>j</sub>y<sub>j</sub> + Σ<sub>i∈I</sub> Σ<sub>j∈J</sub> c<sub>ij</sub>x<sub>ij</sub> 
```

**Constraints:**

1. **Demand satisfaction:** Each customer's demand must be met:

In [71]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (Replace with your actual data)
customers = ['A', 'B', 'C']
facilities = ['1', '2', '3']
fixed_cost = {'1': 100, '2': 150, '3': 120}  
capacity = {'1': 500, '2': 600, '3': 550} 
demand = {'A': 200, 'B': 350, 'C': 250}
transportation_cost = {
    ('A', '1'): 2, ('A', '2'): 3, ('A', '3'): 2.5,
    ('B', '1'): 2.8, ('B', '2'): 1.5, ('B', '3'): 2,
    ('C', '1'): 1.8, ('C', '2'): 2.2, ('C', '3'): 1.7
}

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=customers) # set of customers
model.J = pyo.Set(initialize=facilities) # set of potential facilities

# Parameters
model.f = pyo.Param(model.J, initialize=fixed_cost) # fixed cost of opening a facility
model.cap = pyo.Param(model.J, initialize=capacity) # capacity of a facility
model.d = pyo.Param(model.I, initialize=demand) # demand of each customer
model.c = pyo.Param(model.I, model.J, initialize=transportation_cost) # cost of shipping from i to j

# Variables
mode